# Homework 1
In this homework, we will have you train some convolutional neural networks! We will start with a small dataset, and then work our way up to ImageNet!

For this homework we will train a classifier for Tiny Imagenet which you can read about here https://tiny-imagenet.herokuapp.com/ but don't bother downloading it just yet.

A note on file paths: Last homework, we used the homework1 folder. That was a mistake as it was technically homework 0. You may want to rename the folder from last time to homework0 before you begin.

# Part 0: Initial Setup
You should recognize this code from last time.


In [1]:
# This is code to download and install pytorch
import os
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if os.path.exists('/opt/bin/nvidia-smi') else 'cpu'
!pip install http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

import torch
print('Version', torch.__version__)
print('CUDA enabled:', torch.cuda.is_available())

    100% |████████████████████████████████| 483.0MB 48.8MB/s 
tcmalloc: large alloc 1073750016 bytes == 0x57370000 @  0x7f93e03182a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070
    100% |████████████████████████████████| 61kB 2.4MB/s 
    100% |████████████████████████████████| 2.0MB 5.0MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0
Version 0.4.1
CUDA enabled: True


In [2]:
from google.colab import drive
drive.mount('/gdrive')
!ls /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
'My Drive'


# Part 1: Upload the Dataset
Run this code to navigate to the BASE_PATH directory and upload the homework1.tar file inside the BASE_PATH, then extract it.

Have a look at pt_util. We moved some of the useful functions out of the python notebook to make it less cluttered, and added a few more useful functions.

I made the BASE_PATH and DATA_PATH variables so you don't have to copy the same strings all over the place if you want to move the locations of the files around.


In [3]:
BASE_PATH = '/gdrive/My Drive/colab_files/homework1/'
if not os.path.exists(BASE_PATH):
    os.makedirs(BASE_PATH)
DATA_PATH = BASE_PATH + 'tiny_imagenet/'
PLOT_PATH = BASE_PATH + 'plots/'
if not os.path.exists(PLOT_PATH):
    os.makedirs(PLOT_PATH)


!pwd
!ls
os.chdir(BASE_PATH)
if not os.path.exists(BASE_PATH + DATA_PATH + 'train.h5'):
    !wget http://pjreddie.com/media/files/homework1.tar
    !tar -xvf homework1.tar
    !rm homework1.tar
!ls {DATA_PATH}
os.chdir('/content')

/content
sample_data
--2018-11-03 20:44:04--  http://pjreddie.com/media/files/homework1.tar
Resolving pjreddie.com (pjreddie.com)... 128.208.3.39
Connecting to pjreddie.com (pjreddie.com)|128.208.3.39|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://pjreddie.com/media/files/homework1.tar [following]
--2018-11-03 20:44:05--  https://pjreddie.com/media/files/homework1.tar
Connecting to pjreddie.com (pjreddie.com)|128.208.3.39|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1352140800 (1.3G) [application/octet-stream]
Saving to: ‘homework1.tar’

homework1.tar       100%[===================>]   1.26G  22.0MB/s    in 64s     

2018-11-03 20:45:09 (20.2 MB/s) - ‘homework1.tar’ saved [1352140800/1352140800]

pt_util.py
tiny_imagenet/
tiny_imagenet/train.h5
tiny_imagenet/class_names.txt
tiny_imagenet/val.h5
ls: cannot access '/gdrive/My': No such file or directory
ls: cannot access 'Drive/colab_files/homework1/tiny_imag

In [0]:
import torch
import torchvision
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
import numpy as np
import os
import torch.nn.functional as F
import torch.optim as optim
import h5py
import sys
import matplotlib.pyplot as plt
import math
sys.path.append(BASE_PATH)
import pt_util

# Part 2: Defining the Network
We're giving you no instructions on this part. Welcome to deep learning research! See if you can get above 40% accuracy. You probably want to use the Cross Entropy error again, but who knows, maybe you can find a better loss function. We will give you a few hints of things to try. Have a look at https://github.com/pytorch/examples/blob/master/mnist/main.py for a basic neural network implementation.


- Activation functions other than ReLU
- Batch Norm
- Dropout
- Residual connections

Additionally, the current saving function always saves the latest results. You may want to modify it to only save the results if they have the highest test accuracy. Please make this modification.



In [0]:
class TinyImagenetNet(nn.Module):
    bestacc = 0
    def __init__(self):
        super(TinyImagenetNet, self).__init__()
        
#         # convolutional layers
#         self.conv1 = nn.Conv2d(3, 10, kernel_size=3, stride=2)
#         self.conv2 = nn.Conv2d(10, 30, kernel_size=3, stride=1)
#         self.conv3 = nn.Conv2d(30, 50, kernel_size=3)
#         self.conv4 = nn.Conv2d(50, 100, kernel_size=1)

         # convolutional layers
        c1 = 20
        c2 = 50
        c3 = 200
        c4 = 400
        c5 = 800
        self.conv1 = nn.Conv2d(3, c1, kernel_size=7, stride=1, padding=3)
        self.conv2 = nn.Conv2d(c1, c2, kernel_size=5, stride=1, padding=2)
        self.conv3 = nn.Conv2d(c2, c3, kernel_size=3)
        self.conv4 = nn.Conv2d(c3, c4, kernel_size=1)
        # self.conv5 = nn.Conv2d(c4, c5, kernel_size=1)


        # interleave within convolutional layers
        self.conv1_drop = nn.Dropout2d(0.05)
        self.conv2_drop = nn.Dropout2d(0.05)
        self.conv3_drop = nn.Dropout2d(0.05)
        self.conv4_drop = nn.Dropout2d(0.05)
        self.conv5_drop = nn.Dropout2d(0.05)
        
        # number of features after convolutional layers
        self.linsize = 3600
        
        # also interleave within convolutional layers
        self.bn0 = nn.BatchNorm2d(3, momentum=0.5)
        self.bn1 = nn.BatchNorm2d(c1, momentum=0.5)
        self.bn2 = nn.BatchNorm2d(c2, momentum=0.5)
        self.bn3 = nn.BatchNorm2d(c3, momentum=0.5)
        self.bn4 = nn.BatchNorm2d(c4, momentum=0.5)
        # self.bn5 = nn.BatchNorm2d(c5, momentum=0.5)
        self.bnlin = nn.BatchNorm1d(400, momentum=0.5)
       # self.bnlin2 = nn.BatchNorm1d(250, momentum=0.5)

        
        # run through linear layers
        self.fc1 = nn.Linear(self.linsize, 400)
        self.fc2 = nn.Linear(400, 200)
        # self.fc3 = nn.Linear(250, 200)
        
    def forward(self, x):
      
        batch_size = x.shape[0]
        
        
        # conv layer 1
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(F.max_pool2d(x, 2))
        x = self.conv1_drop(x)

        # conv layer 2
        # 
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(F.max_pool2d(x, 2))
        # x = self.conv2_drop(x)


        # conv layer 3
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(F.max_pool2d(x, 2))
        # x = F.relu(x)
        x = self.conv3_drop(x)

                   
        # conv layer 4
        x = self.conv4(x)
        x = self.bn4(x)
        x = F.relu(F.max_pool2d(x, 2))
        x = F.relu(x)
        # x = self.conv4_drop(x)

        
#         # conv layer 5
#         x = self.conv5(x)
#         x = self.bn5(x)
#         x = F.relu(x)
#         x = self.conv5_drop(x)
#         # x = F.relu(F.max_pool2d(x, 2))
        
        # convert to linear dimensions
        x = x.view(batch_size, self.linsize)

        # linear layer 1
        x = self.fc1(x)
        x = self.bnlin(x)
        x = F.relu(x)
        
        # linear layer 2 (don't batch normarlize) HA SIKE GET WRECKED WE ADDeD A LAYER
        x = self.fc2(x)
       #  x = self.bnlin2(x)
        x = F.relu(x)
        
        #x = self.fc3(x)
        # x = F.relu(x)

        return F.log_softmax(x, dim=1)

    def loss(self, prediction, label, reduction='elementwise_mean'):
        label = label.squeeze()
        loss_val = F.cross_entropy(prediction, label, reduction=reduction)
        return loss_val

    def save_model(self, file_path, num_to_keep=1):
        pt_util.save(self, file_path, num_to_keep)
        
    def save_best_model(self, accuracy, file_path, num_to_keep=1):
        if (bestacc < accuracy):
            bestacc = accuracy
            save_model(file_path) 

    def load_model(self, file_path):
        pt_util.restore(self, file_path)

    def load_last_model(self, dir_path):
        return pt_util.restore_latest(self, dir_path)

This time we are giving you the train and test functions, but feel free to modify them if you want. 

You may need to return some additional information for the logging portion of this assignment.


In [0]:
import time
def train(model, device, train_loader, optimizer, epoch, log_interval):
    model.train()
    
    train_accuracy = 0
    train_loss = 0
    
    for batch_idx, (data, label) in enumerate(train_loader):
        data, label = data.to(device).float(), label.to(device).long()
        optimizer.zero_grad()
        output = model(data)
        loss = model.loss(output, label)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            train_loss = loss.item()
            train_accuracy = 100. * batch_idx / len(train_loader)
            print('{} Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                time.ctime(time.time()),
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
    return train_accuracy, train_loss

def test(model, device, test_loader, return_images=False, log_interval=None):
    model.eval()
    test_loss = 0
    correct = 0

    correct_images = []
    correct_values = []

    error_images = []
    predicted_values = []
    gt_values = []
    with torch.no_grad():
        for batch_idx, (data, label) in enumerate(test_loader):
            data, label = data.to(device).float(), label.to(device).long()
            output = model(data)
            test_loss_on = model.loss(output, label, reduction='sum').item()
            test_loss += test_loss_on
            pred = output.max(1)[1]
            correct_mask = pred.eq(label.view_as(pred))
            num_correct = correct_mask.sum().item()
            correct += num_correct
            if return_images:
                if num_correct > 0:
                    correct_images.append(data[correct_mask, ...].data.cpu().numpy())
                    correct_value_data = label[correct_mask].data.cpu().numpy()[:, 0]
                    correct_values.append(correct_value_data)
                if num_correct < len(label):
                    error_data = data[~correct_mask, ...].data.cpu().numpy()
                    error_images.append(error_data)
                    predicted_value_data = pred[~correct_mask].data.cpu().numpy()
                    predicted_values.append(predicted_value_data)
                    gt_value_data = label[~correct_mask].data.cpu().numpy()[:, 0]
                    gt_values.append(gt_value_data)
            if log_interval is not None and batch_idx % log_interval == 0:
                print('{} Test: [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    time.ctime(time.time()),
                    batch_idx * len(data), len(test_loader.dataset),
                    100. * batch_idx / len(test_loader), test_loss_on))
    if return_images:
        correct_images = np.concatenate(correct_images, axis=0)
        error_images = np.concatenate(error_images, axis=0)
        predicted_values = np.concatenate(predicted_values, axis=0)
        correct_values = np.concatenate(correct_values, axis=0)
        gt_values = np.concatenate(gt_values, axis=0)

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), test_accuracy))
    if return_images:
        return correct_images, correct_values, error_images, predicted_values, gt_values, test_accuracy, test_loss

# PART 3: Loading Data
PyTorch has a nice interface for dealing with a variety of data. You can read a good tutorial here https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
Your friendly neighborhood TAs have made it even easier by preprocessing the data into a nice format. The data you uploaded is stored using hdf5 files which can be acecces a lot like Numpy arrays using the h5py package. In each of the files, there is a "dataset" called 'images', and one called 'labels'. Read more about h5py here http://docs.h5py.org/en/latest/quick.html

Speed hint: With small datasets, it is almost always a good idea to cache the data to disk rather than continually read from files.


In [0]:
# Data loader
class H5Dataset(torch.utils.data.Dataset):
    def __init__(self, h5_file, transform=None):
        # TODO Implement data loading.
        self.f = h5py.File(h5_file, "r")
        self.tset = self.f["images"].value
        self.lset = self.f["labels"].value
        self.transform = transform

    def __len__(self):
        # TODO Implement the length function
        return self.tset.shape[0]
    
    def __getitem__(self, idx):
        # TODO implement the getitem function
        # You should return a tuple of:
        #    a torch tensor containing single image in CxHxW format and
        #    the label as a single tensor scalar.
        data = self.tset[idx]
        label = self.lset[idx]
        if self.transform:
            data = self.transform(data)
        return (data, label)

# Part 4: Dataset Augmentation
In the MNIST assignment, we didn't do any data augmentation because MNIST is kind of easy.

In this assignment, you may find that data augmentation helps you a lot (or possibly hurts your performance).

You can find a bunch preimplemented here https://pytorch.org/docs/stable/torchvision/transforms.html and you can also do your own as seen in the tutorial from part 3.

Play around with various data augmentations we will suggest some.

- ToPILImage - This one is useful for a lot of the built in transforms which expect PIL images. 
- RandomHorizontalFlip
- RandomResizedCrop
- ColorJitter
- RandomRotation
- Normalize
- Adding various types of noise
- ToTensor - PyTorch expects the output from the dataset to be a tensor in CxHxW format.


Note: You should be careful about which of these you apply to the test data. You usually don't want to apply noise to the test data, but you do want to normalize it in the same way for example.


In [0]:
def NHWC_to_NCHW(array):
    return array.transpose((2, 0, 1))

train_transforms = NHWC_to_NCHW
test_transforms = NHWC_to_NCHW

data_train = H5Dataset(DATA_PATH + 'train.h5', transform=train_transforms)
data_test = H5Dataset(DATA_PATH + 'val.h5', transform=test_transforms)

# Part 5: Training the network
Generally, it is useful to see how your training is going. Often people print their loss to make sure it goes down and their accuracy to make sure it goes up. But pictures are better than words. So for this part, you should record and plot the training loss, test loss, and test accuracy (and whatever else you want). 

We have created a very simple logging interface which essentially just saves and restores files via pickle in pt_util. Saving and restoring log data is important if you end your run early and want to continue where you left off rather than starting over.

We have also provided a plot function which can plot a single line graph. You can use it and plot each value independently, or change it to plot them all in one graph. 


__Important note: Do not forget to title your graphs and label your axes. Plots are meaningless without a way to read them.__

Second Note: It will be helpful for you when deciding what network structure, data augmentation, and such work to title the graphs accordingly so you remember.
Third Note: The default setup right now saves and restores the network weights from a single folder. When you modify network architectures, you may want to save the resulting files in different folders (with appropriate names).

We also provided a function for showing some results, because it's not satisfying to train a neural net, you also want to see what it can do! This can also be useful for figuring out what your network is doing well, and what it is failing at. This type of error analysis is very common when training neural networks.


In [0]:
from datetime import datetime

# Play around with these constants, you may find a better setting.
# BATCH_SIZE = 256
# TEST_BATCH_SIZE = 10
# EPOCHS = 200
# LEARNING_RATE = 0.01
# MOMENTUM = 0.9
BATCH_SIZE = 200
TEST_BATCH_SIZE = 500
EPOCHS = 0 #35
LEARNING_RATE = 0.05
MOMENTUM = 0.9
# m = 300
USE_CUDA = True
PRINT_INTERVAL = 100
WEIGHT_DECAY = 0.0005
LOG_PATH = DATA_PATH + 'log.pkl'

# Now the actual training code
use_cuda = USE_CUDA and torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu")
print('Using device', device)
import multiprocessing
print('num cpus:', multiprocessing.cpu_count())

kwargs = {'num_workers': multiprocessing.cpu_count(),
          'pin_memory': True} if use_cuda else {}

class_names = [line.strip().split(', ') for line in open(DATA_PATH + 'class_names.txt')]
name_to_class = {line[1]: line[0] for line in class_names}
class_names = [line[1] for line in class_names]




model = TinyImagenetNet().to(device)

train_loader = torch.utils.data.DataLoader(data_train, batch_size=BATCH_SIZE,
                                           shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(data_test, batch_size=TEST_BATCH_SIZE,
                                          shuffle=False, **kwargs)

print("Data is loaded")

optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
# start_epoch = model.load_last_model(DATA_PATH + 'checkpoints')
start_epoch = 0 # CHANGE THIS

# You may want to define another default for your log data depending on how you save it.
log_data = pt_util.read_log(LOG_PATH, [])

# correct_images, correct_val, error_images, predicted_val, gt_val = test(model, device, test_loader, True)
# correct_images = pt_util.to_scaled_uint8(correct_images.transpose(0, 2, 3, 1))
# error_images = pt_util.to_scaled_uint8(error_images.transpose(0, 2, 3, 1))
# pt_util.show_images(correct_images, ['correct: %s' % class_names[aa] for aa in correct_val])
# pt_util.show_images(error_images, ['pred: %s, actual: %s' % (class_names[aa], class_names[bb]) for aa, bb in zip(predicted_val, gt_val)])

epochs = []
testAcc = []
trainAcc = []
testLoss = []
trainLoss = []

try:
    for epoch in range(start_epoch, EPOCHS + 1):
        print("epoch {}".format(epoch))
        train_accuracy, train_loss = train(model, device, train_loader, optimizer, epoch, PRINT_INTERVAL)
        correct_images, correct_val, error_images, predicted_val, gt_val, test_accuracy, test_loss = test(model, device, test_loader, True)
        # TODO define other things to do at the end of each loop like logging and saving the best model.
        epochs.append(epoch)
        testAcc.append(test_accuracy)
        testLoss.append(test_loss)
        trainAcc.append(train_accuracy)
        trainLoss.append(train_loss)
        


except KeyboardInterrupt as ke:
    print('Interrupted')
except:
    import traceback
    traceback.print_exc()
finally:
    # Always save the most recent model, but don't delete any existing ones.
    model.save_model(DATA_PATH + 'checkpoints/%03d.pt' % epoch, 0)
    
# Show some current correct/incorrect images.
#     correct_images = pt_util.to_scaled_uint8(correct_images.transpose(0, 2, 3, 1))
#     error_images = pt_util.to_scaled_uint8(error_images.transpose(0, 2, 3, 1))
#     pt_util.show_images(correct_images, ['correct: %s' % class_names[aa] for aa in correct_val])
#     pt_util.show_images(error_images, ['pred: %s, actual: %s' % (class_names[aa], class_names[bb]) for aa, bb in zip(predicted_val, gt_val)])
    
  
    plt.subplot(2, 1, 1)
    plt.plot(epochs, testLoss, 'c', label="Test Loss")
    plt.plot(epochs, trainLoss, 'b', label="Train loss")
    plt.title("Loss vs. Epochs")
    plt.xlabel("Epoch Number")
    plt.ylabel("Cross Entropy Loss")
    plt.legend()
    plt.grid(True, color='k')
    
    plt.subplot(2, 1, 2)
    plt.plot(epochs, testAcc, 'r', label="Test Accuracy(%)")
    plt.plot(epochs, trainAcc, 'y', label="Train Accuracy(%)")
    plt.title("Accuracy vs. Epochs")
    plt.xlabel("Epoch Number")
    plt.ylabel("Train Accuracy(%)")
    plt.legend()
    plt.grid(True, color='k')
    plt.savefig(PLOT_PATH + 'test2-{}.pdf'.format(datetime.now()))

# Part 6: ImageNet!
ImageNet is the big granddaddy of supervised vision datasets. It's the dataset that got the whole deep learning train running. We thought it would be fun for you guys to try your hand at it.

A few notes:
- Downloading ImageNet should take about 7 minutues. Extracting should take another 7 minutes. So run this code, then go have a coffee or whatever.
- The data will be removed after 24 hours or so. That's just how Colab works. Each time you reset the environment, you will also have to redownload ImageNet. 
    - I would recommend against storing it on your on Google Drive though as that is way slower to read during training.
    - Through extensive testing, we have found that the fastest way to get the data is the code we provide (faster than saving to google drive, uploading by hand, etc.) 
- Reading images directly with PIL seems to have issues on Colab. Instead try cv2.imread.
    - cv2.imread returns images in bgr order. To go from bgr -> rgb, use    `im = im[:, :, ::-1]`
- The images are provided such that the minimum side length is at least 128 unless the original image was too small for it to be shrunk.
    - transforms.Resize will be your friend
- It seams you cannot use a batch size > 64 x 128 x 128 x 3 while still using multiprocessing due to Colab limitations. You can use larger batches if you set num_workers = 0, but it will be much slower.
- This dataset has 1000 classes, whereas Tiny ImageNet only had 200.
- You will probably want to save your results more than once every epoch since Colab may spurriously shut down your session.
- We know this takes a long time to run. Try leaving it overnight. We don't expect you to get state-of-the-art performance.
- `glob.glob` is a good Python function for reading lots of folder/file names, but it does not return a sorted list.




In [16]:
# Downloads and extracts the data
# !rm -rf 'imagenet128'

if not os.path.exists('imagenet128'):
    !echo $(date +%x_%r)
    !gsutil cp gs://imagenet-cropped/imagenet128.tar .
    !echo $(date +%x_%r) 
    !tar -xf imagenet128.tar
    !echo $(date +%x_%r)

!ls imagenet128/train | head -4

11/03/2018_09:20:44 PM
Copying gs://imagenet-cropped/imagenet128.tar...
\ [1 files][ 23.7 GiB/ 23.7 GiB]   41.6 MiB/s                                   
Operation completed over 1 objects/23.7 GiB.                                     
11/03/2018_09:27:45 PM
11/03/2018_09:33:05 PM
n01440764
n01443537
n01484850
n01491361


In [0]:
# Sets up synset data. Have a look at what is in these dictionaries as they may help you with debugging.
import json
imagenet_synset_data = json.load(open('imagenet128/imagenet_synsets.json'))
synset_id_to_cls = {val['id']: int(key) for key, val in imagenet_synset_data.items()}

In [0]:
# Data loader
import glob
import numpy as np
import cv2
from torchvision import transforms

class ImageFolderLoader(torch.utils.data.Dataset):
    def __init__(self, folder, transform=None):
        # TODO Implement data loading.
        self.folder = folder
        self.idx_to_cls = {}
        self.idx_to_file = {}
        self.transform = transform

        cur_idx = 0
        for folder in glob.iglob(folder + "/*/"):
            synset_id = folder.split("/")[-2]
            for file in glob.iglob(folder + "*"):
                # get synset_id, update idx map
                
                if synset_id not in synset_id_to_cls:
                    print("FAIL... file: {}".format(file))
                else:
                    self.idx_to_cls[cur_idx] = synset_id_to_cls[synset_id]
                    self.idx_to_file[cur_idx] = file
                    cur_idx += 1

                if cur_idx % 10000 == 0:
                    print("loaded {} data points".format(cur_idx))


        
    
    def __len__(self):
        # TODO Implement the length function
        return len(self.idx_to_cls)
    
    def __getitem__(self, idx):
        # TODO implement the getitem function
        # You should return a tuple of:
        #    a torch tensor containing single image in CxHxW format and
        #    the label as a single tensor scalar.
        
        # get numpy representation, update idx map
        file = self.idx_to_file[idx]
        im = cv2.imread(file, cv2.IMREAD_COLOR)
        im = im[:, :, ::-1]

        im = torch.tensor(im.copy())

        if self.transform:
            im = self.transform(im)

        
        label = self.idx_to_cls[idx]
        return (im, label)

In [29]:
from torchvision import transforms

transformer = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

train_transforms = transformer
test_transforms = transformer

data_train = ImageFolderLoader('imagenet128/train', transform=train_transforms)
print(len(data_train))
res = data_train.__getitem__(1)

# assert(len(data_train) == 1281167)
data_test = ImageFolderLoader('imagenet128/val', transform=test_transforms)
# assert(len(data_test) == 50000)

loaded 10000 data points
loaded 20000 data points
loaded 30000 data points
loaded 40000 data points
loaded 50000 data points
loaded 60000 data points
loaded 70000 data points
loaded 80000 data points
loaded 90000 data points
loaded 100000 data points
loaded 110000 data points
loaded 120000 data points
loaded 130000 data points
loaded 140000 data points
loaded 150000 data points
loaded 160000 data points
loaded 170000 data points
loaded 180000 data points
loaded 190000 data points
loaded 200000 data points
loaded 210000 data points
loaded 220000 data points
loaded 230000 data points
loaded 240000 data points
loaded 250000 data points
loaded 260000 data points
loaded 270000 data points
loaded 280000 data points
loaded 290000 data points
loaded 300000 data points
loaded 310000 data points
loaded 320000 data points
loaded 330000 data points
loaded 340000 data points
loaded 350000 data points
loaded 360000 data points
loaded 370000 data points
loaded 380000 data points
loaded 390000 data po

In [0]:
class FullImagenetNet(TinyImagenetNet):
  
    def __init__(self):
      
        super(FullImagenetNet, self).__init__()
         # convolutional layers
        c1 = 20
        c2 = 50
        c3 = 200
        c4 = 400
        c5 = 800
        self.conv1 = nn.Conv2d(3, c1, kernel_size=7, stride=1, padding=3)
        self.conv2 = nn.Conv2d(c1, c2, kernel_size=5, stride=1, padding=2)
        self.conv3 = nn.Conv2d(c2, c3, kernel_size=3)
        self.conv4 = nn.Conv2d(c3, c4, kernel_size=1)
        # self.conv5 = nn.Conv2d(c4, c5, kernel_size=1)


        # interleave within convolutional layers
        self.conv1_drop = nn.Dropout2d(0.05)
        self.conv2_drop = nn.Dropout2d(0.05)
        self.conv3_drop = nn.Dropout2d(0.05)
        self.conv4_drop = nn.Dropout2d(0.05)
        self.conv5_drop = nn.Dropout2d(0.05)
        
        # number of features after convolutional layers
        self.linsize = 19600
        
        # also interleave within convolutional layers
        self.bn0 = nn.BatchNorm2d(3, momentum=0.5)
        self.bn1 = nn.BatchNorm2d(c1, momentum=0.5)
        self.bn2 = nn.BatchNorm2d(c2, momentum=0.5)
        self.bn3 = nn.BatchNorm2d(c3, momentum=0.5)
        self.bn4 = nn.BatchNorm2d(c4, momentum=0.5)
        # self.bn5 = nn.BatchNorm2d(c5, momentum=0.5)
        self.bnlin = nn.BatchNorm1d(2000, momentum=0.5)
       # self.bnlin2 = nn.BatchNorm1d(250, momentum=0.5)

        
        # run through linear layers
        self.fc1 = nn.Linear(self.linsize, 2000)
        self.fc2 = nn.Linear(2000, 1000)
        # self.fc3 = nn.Linear(250, 200)


    def forward(self, x):
        batch_size = x.shape[0]
        
        
        # conv layer 1
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(F.max_pool2d(x, 2))
        x = self.conv1_drop(x)

        # conv layer 2
        # 
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(F.max_pool2d(x, 2))
        # x = self.conv2_drop(x)


        # conv layer 3
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(F.max_pool2d(x, 2))
        # x = F.relu(x)
        x = self.conv3_drop(x)

                   
        # conv layer 4
        x = self.conv4(x)
        x = self.bn4(x)
        x = F.relu(F.max_pool2d(x, 2))
        x = F.relu(x)
        # x = self.conv4_drop(x)

        
#         # conv layer 5
#         x = self.conv5(x)
#         x = self.bn5(x)
#         x = F.relu(x)
#         x = self.conv5_drop(x)
#         # x = F.relu(F.max_pool2d(x, 2))
        
        # convert to linear dimensions
        x = x.view(batch_size, self.linsize)

        # linear layer 1
        x = self.fc1(x)
        x = self.bnlin(x)
        x = F.relu(x)
        
        # linear layer 2 (don't batch normarlize) HA SIKE GET WRECKED WE ADDeD A LAYER
        x = self.fc2(x)
#         x = self.bnlin2(x)
        x = F.relu(x)
        
        #x = self.fc3(x)
        # x = F.relu(x)

        return F.log_softmax(x, dim=1)


In [0]:
from datetime import datetime

BATCH_SIZE = 64
TEST_BATCH_SIZE = 50
EPOCHS = 0
LEARNING_RATE = 0.01
MOMENTUM = 0.9
USE_CUDA = True
PRINT_INTERVAL = 100
WEIGHT_DECAY = 0.0005
CHECKPOINT_PATH = 'imagenet_full/checkpoints'
LOG_PATH = 'imagenet_full/' + 'log.pkl'

use_cuda = USE_CUDA and torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu")
print('Using device', device)
import multiprocessing
print('num cpus:', multiprocessing.cpu_count())

kwargs = {'num_workers': multiprocessing.cpu_count(),
          'pin_memory': True} if use_cuda else {}


class_names = sorted([(int(key), val['label'].split(',')[0]) for key, val in imagenet_synset_data.items()])
name_to_class = {line[1]: line[0] for line in class_names}
class_names = [line[1] for line in class_names]

train_loader = torch.utils.data.DataLoader(data_train, batch_size=BATCH_SIZE,
                                           shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(data_test, batch_size=TEST_BATCH_SIZE,
                                          shuffle=False, **kwargs)

model = FullImagenetNet().to(device)
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
start_epoch = model.load_last_model(CHECKPOINT_PATH)

epochs = []
testAcc = []
trainAcc = []
testLoss = []
trainLoss = []

try:
    for epoch in range(start_epoch, EPOCHS + 1):
        train_accuracy, train_loss = train(model, device, train_loader, optimizer, epoch, PRINT_INTERVAL)
        correct_images, correct_val, error_images, predicted_val, gt_val, test_accuracy, test_loss = test(model, device, test_loader, True)
        model.save_best_model(test_accuracy, CHECKPOINT_PATH + '/%03d.pt' % epoch)

        epochs.append(epoch)
        testAcc.append(test_accuracy)
        testLoss.append(test_loss)
        trainAcc.append(train_accuracy)
        trainLoss.append(train_loss)

except KeyboardInterrupt as ke:
    print('Interrupted')
except:
    import traceback
    traceback.print_exc()
finally:
    model.save_model(CHECKPOINT_PATH + '/%03d.pt' % epoch, 0)
    
    plt.subplot(2, 1, 1)
    plt.plot(epochs, testLoss, 'c', label="Test Loss")
    plt.plot(epochs, trainLoss, 'b', label="Train loss")
    plt.title("Loss vs. Epochs")
    plt.xlabel("Epoch Number")
    plt.ylabel("Cross Entropy Loss")
    plt.legend()
    plt.grid(True, color='k')
    
    plt.subplot(2, 1, 2)
    plt.plot(epochs, testAcc, 'r', label="Test Accuracy(%)")
    plt.plot(epochs, trainAcc, 'y', label="Train Accuracy(%)")
    plt.title("Accuracy vs. Epochs")
    plt.xlabel("Epoch Number")
    plt.ylabel("Train Accuracy(%)")
    plt.legend()
    plt.grid(True, color='k')
    plt.savefig(PLOT_PATH + 'test2-{}.pdf'.format(datetime.now()))

# Part 7: Short answer questions
Please answer these questions, and put the answers in a file called homework1_python.pdf in your repository.

# Tiny ImageNet
1. What design that you tried worked the best? This includes things like network design, learning rate, batch size, number of epochs, and other optimization parameters, data augmentation etc. What was the final train loss? Test loss? Test Accuracy? Provide the plots for train loss, test loss, and test accuracy.
2. What design worked the worst (but still performed better than random chance)? Provide all the same information as question 1.
3. Why do you think the best one worked well and the worst one worked poorly.


# Full ImageNet
1. What design that you tried worked the best? How many epochs were you able to run it for? Provide the same information from Tiny ImageNet question 1.
2. Were you able to use larger/deeper networks on Full ImageNet than you used on Tiny ImageNet and increase accuracy? If so, why? If not, why not?
3. The real ImageNet dataset has significantly larger images. How would you change your network design if the images were twice as large? How about smaller than Tiny ImageNet (32x32)? How do you think your accuracy would change? This is open-ended, but we want a more thought-out answer than "I'd resize the images" or "I'd do a larger pooling stride." You don't have to write code to test your hypothesis.

Additionally, download your .ipynb and put it in your repository and name it homework1_colab.ipynb

